In [16]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import base64  # Added to fix base64 issue
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

from animal_shelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# Username and password and CRUD Python module name
username = "aacuser"
password = "lemonpie"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read_all({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#Added in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Added the HTML image tag and unique identifier into the app.layout
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.H4('Developed by Emma Mitchell for CS-340')),  # Unique identifier
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),  # Center/load logo
    html.Hr(),
    html.Div([
# Code for the interactive filtering options. I used a dropdown menu.
        dcc.Dropdown(
            id='dropdown',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'}, 
                {'label': 'Disaster Rescue', 'value': 'Disaster Rescue'}, 
                {'label': 'Mountain Rescue', 'value': 'Mountain Rescue'}, 
                {'label': 'Reset', 'value': 'Reset'}
            ],
            placeholder='Choose a filter...'
        ),
        html.Div(id='dd-output-container')
        #style = {'font-size':13},
    ]),
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # Features for the data table to make it user-friendly
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
        
    ),
    html.Br(),
    html.Hr(),
# This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('dropdown', 'value')])
def update_dashboard(value):
# Code to filter interactive data table with MongoDB queries

    # Reset returns an unfiltered view of the database
    if value == 'Reset':
        df = pd.DataFrame.from_records(shelter.read_all({}))
    # Returns a filtered view of all avalible dogs best suited for Water Rescue
    elif value == 'Water Rescue':
        df = pd.DataFrame(list(shelter.read_all({"animal_type":"Dog",
                                                "breed":{"$in":["Labrador Retriever Mix",
                                                "Chesapeke Bay Retriever","Newfoundland"]},
                                                "sex_upon_outcome":"Intact Female",
                                                "age_upon_outcome_in_weeks":{"$gte":26},
                                                "age_upon_outcome_in_weeks":{"$lte":156}})))
    # Returns a filtered view of all avalible dogs best suited for Disaster Rescue
    elif value == 'Disaster Rescue':
        df = pd.DataFrame(list(shelter.read_all({"animal_type":"Dog",
                                                "breed":{"$in":["Doberman Pinscher","German Shepherd",
                                                "Golden Retriever","Bloodhound","Rottweiler"]},
                                                "sex_upon_outcome":"Intact Male",
                                                "age_upon_outcome_in_weeks":{"$gte":20},
                                                "age_upon_outcome_in_weeks":{"$lte":300}})))
    # Returns a filtered view of all avalible dogs best suited for Mountain Rescue
    elif value == 'Mountain Rescue':
        df = pd.DataFrame(list(shelter.read_all({"animal_type":"Dog",
                                                "breed":{"$in":["German Shepherd","Alaskan Malamute",
                                                "Old English Sheepdog","Siberian Husky","Rottweiler"]},
                                                "sex_upon_outcome":"Intact Male",
                                                "age_upon_outcome_in_weeks":{"$gte":26},
                                                "age_upon_outcome_in_weeks":{"$lte":156}})))


    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')


    return (data,columns)


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Chart code. I was unable to impliment this into the final product due to unresolved issues. I have documented
# everything I tried to resolve the issue.

# @app.callback(
#     Output('graph-id', "children"),
#     [Input('datatable-id', "derived_viewport_data")])
# def update_graphs(viewData):
#     # Pie Chart code [Error: doesn't display data, just an empty space and the title]
#     # Tried switching pd to px as suggested in documentation, but this did not fix the issue.
#     # Tried using fig.show() as suggested here (https://plotly.com/python/pie-charts/) did not fix the issue
#     # Tried following the documentation from (https://dash.plotly.com/dash-core-components/graph) didn't work
#
#     df = pd.DataFrame.from_dict(viewData)
#     fig = px.pie(df, values='breed', names='breed', title='Avalible Breeds')
#     return [
#        dcc.Graph(figure=fig)   
#     ]

# Map code
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
# Code for your geolocation chart
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
# Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]



app